In [61]:
import pandas as pd
import numpy as np
import datetime
import math
from tqdm.notebook import tqdm

In [62]:
from google.colab import drive
drive.mount('/content/Data/')

Drive already mounted at /content/Data/; to attempt to forcibly remount, call drive.mount("/content/Data/", force_remount=True).


In [63]:
path_to_features = '/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Features/'

Ссылки на файлы:
https://drive.google.com/drive/folders/1PiuDeB1iyrVn-N4OdnvdO8a00aRnauQe?usp=sharing

In [64]:
def read_tables():
  feat_table = pd.read_csv(path_to_features + 'features_2020_1.csv', index_col = 0)
  for i in tqdm(range(2,13)):
    feat_table = feat_table.append(pd.read_csv(path_to_features + f'features_2020_{i}.csv',index_col = 0))
  for i in tqdm(range(1,13)):
    feat_table = feat_table.append(pd.read_csv(path_to_features + f'3dfeatures_2020_{i}.csv', index_col = 0))
  for i in tqdm(range(1,13)):
    feat_table = feat_table.append(pd.read_csv(path_to_features + f'freefeatures_2020_{i}.csv', index_col = 0))
  feat_table = feat_table.drop(feat_table[feat_table['2 metre temperature'] == '--'].index).reset_index(drop=True)
  return feat_table.sample(feat_table.shape[0]).astype(np.float32) # рандомим строки


In [65]:
feat_table = read_tables()
print(feat_table.dtypes)
feat_table.head()

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

lat                                 float32
lon                                 float32
lat_idx                             float32
lon_idx                             float32
10 metre U wind component           float32
10 metre V wind component           float32
2 metre temperature                 float32
Leaf area index, high vegetation    float32
Leaf area index, low vegetation     float32
Skin reservoir content              float32
Skin temperature                    float32
Snow cover                          float32
Surface latent heat flux            float32
Surface net solar radiation         float32
Surface sensible heat flux          float32
Evaporation                         float32
Total precipitation                 float32
0d_fire                             float32
1d_fire                             float32
2d_fire                             float32
dtype: object


,lat,lon,lat_idx,lon_idx,10 metre U wind component,10 metre V wind component,2 metre temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Skin reservoir content,Skin temperature,Snow cover,Surface latent heat flux,Surface net solar radiation,Surface sensible heat flux,Evaporation,Total precipitation,0d_fire,1d_fire,2d_fire
9406,64.400002,164.199997,171.0,1452.0,-1.009109,-0.228775,278.260742,2.054077,1.244019,3.188848e-06,273.714600,0.0,-4622366.0,13822584.0,-4325897.0,-0.001848,0.000689,9.0,9.0,11.0
15598,58.099998,121.400002,234.0,1024.0,-0.877197,1.114929,279.133545,1.374146,0.000000,-4.640386e-20,277.549316,0.0,-2167140.0,11115600.0,-2284099.0,-0.000867,0.000000,9.0,9.0,9.0
37757,66.900002,92.400002,146.0,734.0,-1.569763,0.267181,292.285889,0.000000,1.244751,1.102686e-06,291.000977,0.0,-5965656.0,14786190.0,-1134028.0,-0.002385,0.000029,11.0,11.0,11.0
6640,55.000000,79.800003,265.0,608.0,-2.333832,0.807220,290.219604,1.689941,2.321411,-4.336809e-20,289.599121,0.0,-8688120.0,20888812.0,-3364256.0,-0.003474,0.000000,6.0,11.0,6.0
17031,62.900002,135.300003,186.0,1163.0,-1.119614,-1.132248,278.803711,1.338501,0.000000,1.400709e-06,275.700684,0.0,-1199556.0,9057656.0,-2500626.0,-0.000480,0.000000,8.0,9.0,8.0


In [66]:
# поменяем теперь классы пожаров на то был ли пожар или нет.
for i in tqdm(range(feat_table.shape[0])):
  for j in range(3):
    if feat_table.at[i,f'{j}d_fire'] == 11:
      feat_table.at[i,f'{j}d_fire'] = 0
    else:
      feat_table.at[i,f'{j}d_fire'] = 1
feat_table

  0%|          | 0/40758 [00:00<?, ?it/s]

,lat,lon,lat_idx,lon_idx,10 metre U wind component,10 metre V wind component,2 metre temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Skin reservoir content,Skin temperature,Snow cover,Surface latent heat flux,Surface net solar radiation,Surface sensible heat flux,Evaporation,Total precipitation,0d_fire,1d_fire,2d_fire
9406,64.400002,164.199997,171.0,1452.0,-1.009109,-0.228775,278.260742,2.054077,1.244019,3.188848e-06,273.714600,0.0,-4622366.0,13822584.0,-4325897.0,-0.001848,0.000689,1.0,1.0,0.0
15598,58.099998,121.400002,234.0,1024.0,-0.877197,1.114929,279.133545,1.374146,0.000000,-4.640386e-20,277.549316,0.0,-2167140.0,11115600.0,-2284099.0,-0.000867,0.000000,1.0,1.0,1.0
37757,66.900002,92.400002,146.0,734.0,-1.569763,0.267181,292.285889,0.000000,1.244751,1.102686e-06,291.000977,0.0,-5965656.0,14786190.0,-1134028.0,-0.002385,0.000029,0.0,0.0,0.0
6640,55.000000,79.800003,265.0,608.0,-2.333832,0.807220,290.219604,1.689941,2.321411,-4.336809e-20,289.599121,0.0,-8688120.0,20888812.0,-3364256.0,-0.003474,0.000000,1.0,0.0,1.0
17031,62.900002,135.300003,186.0,1163.0,-1.119614,-1.132248,278.803711,1.338501,0.000000,1.400709e-06,275.700684,0.0,-1199556.0,9057656.0,-2500626.0,-0.000480,0.000000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18779,51.799999,41.500000,297.0,225.0,-3.059372,3.167553,289.256805,2.289795,1.315063,-4.813858e-20,289.809387,0.0,-452536.0,9383320.0,-2956732.0,-0.000181,0.000000,1.0,0.0,0.0
16156,53.799999,40.500000,277.0,215.0,3.008087,4.059113,293.830566,2.239502,1.671875,-9.020570e-20,294.577881,0.0,-3248842.0,13463154.0,-3646632.0,-0.001299,0.000000,1.0,0.0,0.0
9664,60.799999,154.500000,207.0,1355.0,0.104538,-0.710251,287.381348,0.000000,2.781372,7.271767e-06,281.828613,0.0,-6493904.0,12985460.0,-1357976.0,-0.002597,0.000972,0.0,0.0,0.0
18973,59.599998,143.000000,219.0,1240.0,1.548737,-2.256256,275.510010,1.728271,1.172729,-5.030699e-20,275.316040,0.0,-1186756.0,6805418.0,85728.0,-0.000475,0.000000,1.0,1.0,0.0


In [67]:
print(f"Part of verified fires on 0 day {feat_table.loc[feat_table['0d_fire'] == 1]['0d_fire'].count() / len(feat_table):.2f}")
print(f"Part of verified fires on 1 day {feat_table.loc[feat_table['1d_fire'] == 1]['1d_fire'].count() / len(feat_table):.2f}")
print(f"Part of verified fires on 2 day {feat_table.loc[feat_table['2d_fire'] == 1]['2d_fire'].count() / len(feat_table):.2f}")
# относительно сбалансированно

Part of verified fires on 0 day 0.58
Part of verified fires on 1 day 0.32
Part of verified fires on 2 day 0.37


In [68]:
# Нормализуем данные
norm_ftable = feat_table.iloc[:,4:-3] # координаты мне не нужны
norm_ftable=(norm_ftable-norm_ftable.min())/(norm_ftable.max()-norm_ftable.min())
norm_ftable.head()

,10 metre U wind component,10 metre V wind component,2 metre temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Skin reservoir content,Skin temperature,Snow cover,Surface latent heat flux,Surface net solar radiation,Surface sensible heat flux,Evaporation,Total precipitation
9406,0.398064,0.389815,0.613467,0.366649,0.290997,4.007641e-03,0.531953,0.0,0.574008,0.546544,0.459541,0.585580,0.025819
15598,0.403423,0.436041,0.623109,0.245283,0.000000,6.267926e-17,0.571972,0.0,0.711188,0.439510,0.570540,0.725525,0.000000
37757,0.375287,0.406877,0.768403,0.000000,0.291168,1.385820e-03,0.712353,0.0,0.498955,0.584645,0.633062,0.509014,0.001077
6640,0.344246,0.425455,0.745577,0.301652,0.543017,6.649451e-17,0.697723,0.0,0.346844,0.825943,0.511819,0.353836,0.000000
17031,0.393574,0.358733,0.619465,0.238920,0.000000,1.760366e-03,0.552680,0.0,0.765249,0.358139,0.558769,0.780676,0.000000


In [69]:
display(norm_ftable.head())
norm_ftable.corr()

,10 metre U wind component,10 metre V wind component,2 metre temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Skin reservoir content,Skin temperature,Snow cover,Surface latent heat flux,Surface net solar radiation,Surface sensible heat flux,Evaporation,Total precipitation
9406,0.398064,0.389815,0.613467,0.366649,0.290997,4.007641e-03,0.531953,0.0,0.574008,0.546544,0.459541,0.585580,0.025819
15598,0.403423,0.436041,0.623109,0.245283,0.000000,6.267926e-17,0.571972,0.0,0.711188,0.439510,0.570540,0.725525,0.000000
37757,0.375287,0.406877,0.768403,0.000000,0.291168,1.385820e-03,0.712353,0.0,0.498955,0.584645,0.633062,0.509014,0.001077
6640,0.344246,0.425455,0.745577,0.301652,0.543017,6.649451e-17,0.697723,0.0,0.346844,0.825943,0.511819,0.353836,0.000000
17031,0.393574,0.358733,0.619465,0.238920,0.000000,1.760366e-03,0.552680,0.0,0.765249,0.358139,0.558769,0.780676,0.000000


,10 metre U wind component,10 metre V wind component,2 metre temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Skin reservoir content,Skin temperature,Snow cover,Surface latent heat flux,Surface net solar radiation,Surface sensible heat flux,Evaporation,Total precipitation
10 metre U wind component,1.000000,-0.023087,-0.212058,-0.080507,0.063302,0.149315,-0.216742,0.194902,0.138290,-0.221890,0.209676,0.140242,0.105658
10 metre V wind component,-0.023087,1.000000,0.055076,0.019016,0.034347,0.040965,0.041199,0.054703,0.019408,-0.101402,0.280989,0.019122,-0.015739
2 metre temperature,-0.212058,0.055076,1.000000,0.381119,0.034700,-0.452588,0.990898,-0.764378,-0.656299,0.744749,-0.479644,-0.658716,0.034319
"Leaf area index, high vegetation",-0.080507,0.019016,0.381119,1.000000,0.043042,-0.090663,0.373637,-0.300390,-0.404971,0.304379,-0.140571,-0.405792,0.021096
"Leaf area index, low vegetation",0.063302,0.034347,0.034700,0.043042,1.000000,0.016124,0.025516,-0.058213,-0.110466,0.058262,0.054483,-0.109572,0.000125
Skin reservoir content,0.149315,0.040965,-0.452588,-0.090663,0.016124,1.000000,-0.441680,0.683833,0.271007,-0.447394,0.376973,0.274524,0.372250
Skin temperature,-0.216742,0.041199,0.990898,0.373637,0.025516,-0.441680,1.000000,-0.759339,-0.635902,0.722801,-0.487143,-0.638547,0.040498
Snow cover,0.194902,0.054703,-0.764378,-0.300390,-0.058213,0.683833,-0.759339,1.000000,0.503844,-0.652238,0.507471,0.509503,0.031153
Surface latent heat flux,0.138290,0.019408,-0.656299,-0.404971,-0.110466,0.271007,-0.635902,0.503844,1.000000,-0.781924,0.288842,0.999859,-0.041460
Surface net solar radiation,-0.221890,-0.101402,0.744749,0.304379,0.058262,-0.447394,0.722801,-0.652238,-0.781924,1.000000,-0.724825,-0.781634,-0.127217


In [70]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df):
    au_corr = df.copy().corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr

def remove_correlated_rows(df, threshold):
  level = 1
  df = df.copy()
  while(True):
    temp = get_top_abs_correlations(df).reorder_levels([1, 0])
    for i,j in temp.items():
      if j < threshold:
        level = j
        break
      print('Get rid of ',i[0])
      df = df.drop(columns=[i[0]])
      break
    if level < threshold:
      break
  return df

temp = get_top_abs_correlations(norm_ftable).reorder_levels([1, 0])
print("We have to do something with correlated data\n\n", temp.loc[temp>0.4])

We have to do something with correlated data

 Evaporation                  Surface latent heat flux            0.999859
Skin temperature             2 metre temperature                 0.990898
Surface net solar radiation  Surface latent heat flux            0.781924
Evaporation                  Surface net solar radiation         0.781634
Snow cover                   2 metre temperature                 0.764378
                             Skin temperature                    0.759339
Surface net solar radiation  2 metre temperature                 0.744749
Surface sensible heat flux   Surface net solar radiation         0.724825
Surface net solar radiation  Skin temperature                    0.722801
Snow cover                   Skin reservoir content              0.683833
Evaporation                  2 metre temperature                 0.658716
Surface latent heat flux     2 metre temperature                 0.656299
Surface net solar radiation  Snow cover                          

In [71]:
uncor_df = remove_correlated_rows(norm_ftable, 0.6)
uncor_columns = uncor_df.columns
display(uncor_df.corr())
print(uncor_columns)
temp = get_top_abs_correlations(uncor_df)
temp.loc[temp > 0.2]

Get rid of  Evaporation
Get rid of  Skin temperature
Get rid of  Surface net solar radiation
Get rid of  Snow cover
Get rid of  Surface latent heat flux


,10 metre U wind component,10 metre V wind component,2 metre temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Skin reservoir content,Surface sensible heat flux,Total precipitation
10 metre U wind component,1.000000,-0.023087,-0.212058,-0.080507,0.063302,0.149315,0.209676,0.105658
10 metre V wind component,-0.023087,1.000000,0.055076,0.019016,0.034347,0.040965,0.280989,-0.015739
2 metre temperature,-0.212058,0.055076,1.000000,0.381119,0.034700,-0.452588,-0.479644,0.034319
"Leaf area index, high vegetation",-0.080507,0.019016,0.381119,1.000000,0.043042,-0.090663,-0.140571,0.021096
"Leaf area index, low vegetation",0.063302,0.034347,0.034700,0.043042,1.000000,0.016124,0.054483,0.000125
Skin reservoir content,0.149315,0.040965,-0.452588,-0.090663,0.016124,1.000000,0.376973,0.372250
Surface sensible heat flux,0.209676,0.280989,-0.479644,-0.140571,0.054483,0.376973,1.000000,0.124477
Total precipitation,0.105658,-0.015739,0.034319,0.021096,0.000125,0.372250,0.124477,1.000000


Index(['10 metre U wind component', '10 metre V wind component',
       '2 metre temperature', 'Leaf area index, high vegetation',
       'Leaf area index, low vegetation', 'Skin reservoir content',
       'Surface sensible heat flux', 'Total precipitation'],
      dtype='object')


2 metre temperature        Surface sensible heat flux          0.479644
                           Skin reservoir content              0.452588
                           Leaf area index, high vegetation    0.381119
Skin reservoir content     Surface sensible heat flux          0.376973
                           Total precipitation                 0.372250
10 metre V wind component  Surface sensible heat flux          0.280989
10 metre U wind component  2 metre temperature                 0.212058
                           Surface sensible heat flux          0.209676
dtype: float64

In [99]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [73]:
class MultipleClassification(BaseEstimator, RegressorMixin):

  def __init__(self):
    '''
    params_size : Размер входной таблицы параметров
    answers_size : Количество предсказываемых параметров
    '''
    self.params_size = 0 
    self.answer_size = 0

  def fit(self,X, y):
    self.params_size = X.shape[1]
    self.answer_size = y.shape[1]
    
    self.__check_data__(X,y) # Проверяем некотурую корректность данных
    # Для каждого дня создаем отдельный классификатор
    self.classifiers = [GradientBoostingClassifier() for i in range(self.answer_size)] # зададим список классификаторов для каждого дня

    y = y.reset_index(drop=True)
    if type(X) == np.ndarray:
      X = pd.DataFrame(X)
    else:
      X = X.copy() # Копируем, так как будем изменять

    # можно попробовать sklearn.multioutput.MultiOutputClassifier, но не думаю, что он будет учитывать корреляцию между днями
    for i in range(self.answer_size):
      self.classifiers[i].fit(X,y.iloc[:,i]) # тренируем опознавание дня с учетом предыдущих дней
      X[f'{i}d_fire'] = pd.Series(y.iloc[:,i], index=X.index) # добавляю новый столбец, чтобы учить следующий день

  def score(self,X,y):
    self.__check_data__(X,y)

    y = y.reset_index(drop=True)

    if type(X) == np.ndarray:
      X = pd.DataFrame(X)
    else:
      X = X.copy()

    scores_for_days = []
 
    for i in range(self.answer_size):
      # тренируем опознавание дня с учетом предыдущих дней
      scores_for_days.append(self.classifiers[i].score(X,y.iloc[:,i]))
       # добавляю новый столбец, чтобы предсказать следующий день
      X[f'{i}d_fire'] = pd.Series(self.classifiers[i].predict(X), index=X.index)
 
    return scores_for_days

  def predict(self,X):
    self.__check_data__(X)

    if type(X) == np.ndarray:
      X = pd.DataFrame(X)
    else:
      X = X.copy()

    scores_for_days = []

    for i in range(self.answer_size):
       # добавляю новый столбец, чтобы предсказать следующий день
       # Добавляю в виде столбца результат прогноза предыдущего дня
      X[f'{i}d_fire'] = pd.Series(self.classifiers[i].predict(X), index=X.index)

    return X

  def __check_data__(self,X,y = None):
    if(X.shape[1] != self.params_size): 
      raise ValueError('Expected X shape(n,{}) got (n,{})'.format(self.params_size, X.shape[1]))
    
    if(y is None):
      return

    if y.shape[1] != self.answer_size:
      raise ValueError('Expected y shape(n,{}) got (n,{})'.format(self.answer_size, y.shape[1]))

    if(X.shape[0] != y.shape[0]):
      raise ValueError('Different amount of rows at X and y ({}!={})'.format(X.shape[0],y.shape[0]))

In [74]:
pipl = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', MultipleClassification())
])

X_train, X_test, Y_train, Y_test = train_test_split(feat_table[uncor_columns], feat_table.iloc[:,-3:], test_size=0.2, random_state=10)

In [75]:
pipl.fit(X_train,Y_train)

Pipeline(memory=None,
         steps=[('scaling',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regression', MultipleClassification())],
         verbose=False)

In [76]:
pipl.score(X_test,Y_test)

[0.6795878312070658, 0.7520853778213935, 0.6805691854759568]

In [77]:
y_tested = pipl.predict(X_test)

In [97]:
print(f1_score(Y_test.reset_index(drop=True),y_tested.iloc[:,-3:], average = None))

[0.75404896 0.61658129 0.54665738]


In [100]:
print(roc_auc_score(Y_test.reset_index(drop=True),y_tested.iloc[:,-3:], average = None))
# Результаты так себе, но в качестве Baseline решения пойдет

[0.64795346 0.71756123 0.64684861]
